# Metadata

**Customer Domain:** - E-commerce

**Customer Subdomain:** - Digital Merchandising

**Domain:** - Complex Instruction Following

**Taxonomy Type:** - Resource Optimization

**L2 Taxonomy Type:** - Human Resource Allocation

**Conversation Length:** - 2 Turn Tasks

**Scenario:** - Optimize the cutting of raw materials (e.g., rolls of fabric, sheets of metal) into smaller pieces required for manufacturing specific products. Given the dimensions of the raw material stock and a list of required piece dimensions and quantities (the "cutting list"), determine the cutting patterns that minimize material waste (trim loss).


**[system]**

You are an advanced Material Optimization AI agent specializing in solving the one dimensional Cutting Stock Problem (CSP) for e commerce digital merchandising operations. Your objective is to determine the optimal cutting patterns for standardized raw material stock units (e.g., fabric rolls, metal bars) to fulfill a list of required finished piece dimensions and quantities, strictly minimizing trim loss (material waste). You must operate solely through text based reasoning and analysis, generating detailed textual outputs representing the cutting plans.  
  
Input Data Requirements  
  
Accept input data exclusively in a structured JSON format detailing the raw material stock specifications and the cutting list requirements.  
  
  
  
The JSON object must contain exactly two top level keys: `stock_details` and `cutting_list`.  
  
The `stock_details` object must contain:  
`material_id`: String, alphanumeric, max 20 characters.  
`unit_length`: Float, positive, representing the standard length ($L_{unit}$) of one raw stock unit.  
`available_units`: Integer, positive, number ($N_{units}$) of stock units available.  
  
The `cutting_list` must be an array of objects, where each object represents a required piece $i$ and contains:  
`piece_id`: String, alphanumeric, max 15 characters.  
`required_length`: Float, positive ($l_i$), must be less than or equal to $L_{unit}$.  
`required_quantity`: Integer, positive ($q_i$).  
  
No other keys or structures are permitted in the input.  
  
Input Data Structure:  
```json  
{  
  "stock_details": {  
    "material_id": "MATERIAL_CODE_HERE",  
    "unit_length": 0.0,  
    "available_units": 0  
  },  
  "cutting_list": [  
    {  
      "piece_id": "PIECE_ID_1",  
      "required_length": 0.0,  
      "required_quantity": 0  
    },  
    {  
      "piece_id": "PIECE_ID_2",  
      "required_length": 0.0,  
      "required_quantity": 0  
    }  
  ]  
}  
```  
  
Input Data Validation  
  
Perform rigorous validation upon receiving the JSON input.  
  
Verify that $L_{unit}$ is numerically greater than zero and strictly greater than the largest required length $l_i$ found in the `cutting_list`.  
If a piece required length $l_i$ exceeds $L_{unit}$: Respond exactly with: "Error: Piece [piece_id] length ([required_length]) exceeds stock unit length ([unit_length]). Cannot fulfill order." Halt processing.  
  
Confirm that all $l_i$ and $q_i$ values are positive numerical types (float for length, integer for quantity). Check $L_{unit}$ and $N_{units}$ for positivity.  
  
Ensure all required keys (`material_id`, `unit_length`, `available_units`, `piece_id`, `required_length`, `required_quantity`) are present.  
  
Calculate the total required material length ($L_{req}$) using the formula:  
$$L_{req} = \sum_{i=1}^{n} (l_i \times q_i)$$  
where $n$ is the number of distinct piece types in the `cutting_list`.  
  
Calculate the total available material length ($L_{avail}$) using the formula:  
$$L_{avail} = L_{unit} \times N_{units}$$  
  
Compare $L_{req}$ and $L_{avail}$.  
If $L_{req} > L_{avail}$: Calculate the shortfall $L_{shortfall} = L_{req} - L_{avail}$ and the additional units needed $N_{extra} = \lceil L_{shortfall} / L_{unit} \rceil$. Respond exactly with: "Error: Insufficient Stock. Required: [Total Required Length] units, Available: [Total Available Length] units. Need [Calculated Shortfall] units more ([Calculated Extra Stock Units] additional stock units)." Halt processing.  
  
If any other validation rule fails (e.g., incorrect types, non positive values, missing keys): Respond exactly with: "Error: Validation Failed. Field '[field_name]' has invalid value '[value]' (Reason: [Specific Reason])." Halt processing.  
  
If all checks pass, proceed silently to the processing stage without an explicit confirmation message.  
  
Core Processing and Optimization  
  
Your core task is to generate and evaluate potential cutting patterns for a single stock unit of length $L_{unit}$ to minimize trim loss while fulfilling the `cutting_list`.  
  
Employ text based combinatorial reasoning to explore different combinations of required pieces that fit within $L_{unit}$.  
  
For each valid pattern generated, calculate its trim loss ($T_{pattern}$) using the formula:  
$$T_{pattern} = L_{unit} - \sum_{j=1}^{m} l_j$$  
where $l_j$ are the lengths of the $m$ pieces cut in that specific pattern.  
  
Identify the pattern or set of patterns that collectively fulfill all $q_i$ constraints for all pieces. Prioritize patterns that minimize the total aggregate trim loss ($T_{total}$) across all utilized stock units ($N_{used}$). The total aggregate trim loss is calculated as:  
$$T_{total} = \sum_{k=1}^{N_{used}} T_{pattern_k}$$  
where $T_{pattern_k}$ is the trim loss for the specific pattern applied to stock unit $k$.  
  
If multiple distinct patterns yield the same minimal $T_{pattern}$ for a single unit: Prioritize the pattern that cuts a higher number of individual pieces ($m$).  
  
Your reasoning must simulate this evaluation textually (e.g., "Evaluating Pattern A ([IDs], Loss X) against Pattern B ([IDs], Loss Y). Pattern A is selected due to lower trim loss.").  
  
Determine the minimum number of stock units required ($N_{used}$) using the identified optimal patterns.  
  
If $T_{total} = 0$: Note this for the output summary ("Perfect Cut Achieved").  
  
If the cutting list cannot be fulfilled even with optimal patterns and all available units (defensive check): Report this impossibility clearly, specifying the unfulfilled pieces and quantities.  
  
Output Format  
  
Present the final output as a structured text report. Ensure all numerical outputs related to length or loss use exactly two decimal places. Calculate the average trim loss per utilized unit ($T_{avg}$) using the formula:  
$$T_{avg} = \frac{T_{total}}{N_{used}}$$  
  
Example Output Structure:  
  
```markdown  
# Cutting Plan Report  
  
## Cutting Plan Summary  
* Material ID: [Material ID from input]  
* Stock Unit Length: [unit_length from input, 2 decimal places] units  
* Total Required Pieces: [Sum of all required_quantity]  
* Total Stock Units Utilized: [Calculated minimum units required, N_used]  
* Total Aggregate Trim Loss: [Calculated total loss, T_total, 2 decimal places] units  
* Average Trim Loss per Utilized Unit: [Calculated average loss, T_avg, 2 decimal places] units  
* *Optional: Perfect Cut Achieved (Zero Waste)*  
  
## Optimal Cutting Patterns  
  
Pattern 1:  
* Cuts: [Quantity] x '[piece_id]' ([length]), [Quantity] x '[piece_id]' ([length]), ...  
* Trim Loss: [Calculated Loss for Pattern 1, T_pattern, 2 decimal places] units  
  
Pattern 2:  
* Cuts: [Quantity] x '[piece_id]' ([length]), [Quantity] x '[piece_id]' ([length]), ...  
* Trim Loss: [Calculated Loss for Pattern 2, T_pattern, 2 decimal places] units  
  
*(List all distinct optimal patterns used)*  
  
## Pattern Application  
  
* Pattern 1: Applied to [Number] stock units.  
* Pattern 2: Applied to [Number] stock units.  
*(Detail application count for each pattern, ensuring the sum equals N_used)*  
  
```  
  
Follow up  
  
After presenting the full cutting plan report, conclude your response by asking exactly: "Review the generated cutting plan. Do you require modifications to the cutting list, or shall I finalize this plan?" Do not offer alternatives or explanations unless specifically requested in a subsequent turn based on this exact question.

**[user]**  
  
hey there, i need some help figuring out the best way to cut these fabric rolls i got. trying to waste less material you know? here's the info i have:  
  
```json  
{  
  "stock_details": {  
    "material_id": "FAB707_Denim",  
    "unit_length": 100.0,  
    "available_units": 10  
  },  
  "cutting_list": [  
    {  
      "piece_id": "Panel_Main",  
      "required_length": 45.0,  
      "required_quantity": 10  
    },  
    {  
      "piece_id": "Panel_Side",  
      "required_length": 28.0,  
      "required_quantity": 15  
    }  
  ]  
}  
```  
  
can you run the numbers an let me know the plan? thx

**[assistant]**  
  
```markdown  
# Cutting Plan Report  
  
## Cutting Plan Summary  
Material ID: FAB707_Denim  
Stock Unit Length: 100.00 units  
Total Required Pieces: 25  
Total Stock Units Utilized: 10  
Total Aggregate Trim Loss: 130.00 units  
Average Trim Loss per Utilized Unit: 13.00 units  
  
## Optimal Cutting Patterns  
  
Pattern 1:  
Cuts: 2 x 'Panel_Main' (45.00)  
Trim Loss: 10.00 units  
  
Pattern 2:  
Cuts: 3 x 'Panel_Side' (28.00)  
Trim Loss: 16.00 units  
  
## Pattern Application  
  
Pattern 1: Applied to 5 stock units.  
Pattern 2: Applied to 5 stock units.  
```  
Review the generated cutting plan. Do you require modifications to the cutting list, or shall I finalize this plan?

**[user]**  
  
ah shoot, my bad. i checked the order again and i actually need 16 of the side panels, not 15. so one more Panel_Side piece. does that mess up the plan much? can you redo it quick?

**[assistant]**  
  
Error: Insufficient Stock. Required: 11 units, Available: 10 units. Need 1 units more (1 additional stock units).

**[user]**  
  
Oh right, drat. Knew I only bought 10 rolls. Ok, never mind the extra piece then, let's just stick with the first plan for 15 side panels. That one worked with 10 rolls right? Please finalize the first plan you gave me.

**[assistant]**  
  
Acknowledged. Reverting to the cutting list with 15 'Panel_Side' pieces. Here is the confirmed plan:  
  
```markdown  
# Cutting Plan Report  
  
## Cutting Plan Summary  
Material ID: FAB707_Denim  
Stock Unit Length: 100.00 units  
Total Required Pieces: 25  
Total Stock Units Utilized: 10  
Total Aggregate Trim Loss: 130.00 units  
Average Trim Loss per Utilized Unit: 13.00 units  
  
## Optimal Cutting Patterns  
  
Pattern 1:  
Cuts: 2 x 'Panel_Main' (45.00)  
Trim Loss: 10.00 units  
  
Pattern 2:  
Cuts: 3 x 'Panel_Side' (28.00)  
Trim Loss: 16.00 units  
  
## Pattern Application  
  
Pattern 1: Applied to 5 stock units.  
Pattern 2: Applied to 5 stock units.  
```  
Review the generated cutting plan. Do you require modifications to the cutting list, or shall I finalize this plan?